# Weighted Voronoi Stippling (V1)

In [110]:
from PIL import Image, ImageDraw
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
import numpy as np
import time

from common_functions import Closest_point, Raster_Centroid

In [111]:
image_resolution = 500
image_path = "sampling/test_im_2_500.png"
point_count = 250
res = 150
threshold = 255

scale_factor = image_resolution / res #scale factor between sample resolution and image resolution
sample_offset = 0.5 * scale_factor #offset so the pixels are sampled correctly

final_resolution = 1080

In [112]:
def Weighted_Raster_Centroid(region,image_path,resolution):

    image = Image.open(image_path)
    cx = 0
    cy = 0
    total_weight = 0
    for pixel in region:
        sampled_value = image.getpixel((
            pixel[0] * scale_factor + sample_offset,
            pixel[1] * scale_factor + sample_offset
            ))
        # print(pixel)
        weight = 1 - np.mean(sampled_value) / 255
        if weight < 0.01:
            weight = 0.01
        total_weight += weight
         
        cy += pixel[1] * weight
        cx += pixel[0] * weight
        # cx += pixel[0] * weight
        # cy += pixel[1] * weight

    cx /= total_weight
    cy /= total_weight

    ## Random jitter
    jitter_mag = .75
    cx += (np.random.rand() - 0.5) * jitter_mag
    cy += (np.random.rand() - 0.5) * jitter_mag

    if cx == 0.5 or cy == 0.5:
        print(True)
    
    return [cx, cy]

In [113]:
def KD_closest_point(search,points,tree):

    result = tree.query(search,k=1,workers=-1)
    closest = points[result[1]]

    return closest

In [114]:
sample_locations = []
for x in range(res):
    for y in range(res):
        search = [x,y]
        sample_locations.append(search)

Option 1: Random points

In [115]:
# seeds = np.random.rand(point_count,2) * res
# startseeds = seeds
# print(seeds)

Option 2: Rejection sampling

In [116]:
image = Image.open(image_path)

seeds = []
startseeds = seeds
n = 0
while n < point_count:
    rand_x = np.random.rand()
    rand_y = np.random.rand()
    img_x = rand_x * image_resolution
    img_y = rand_y * image_resolution
    pixel = np.max(image.getpixel((img_x,img_y)))

    rand_x = rand_x * res
    rand_y = rand_y * res
    
    if pixel < threshold:
        seed = np.array([rand_x,rand_y])
        seeds.append(seed)
        n += 1
    else:
        continue

def center_ellipse(x,y,r,c):
    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
offset = 0.5
draw.rectangle([0,0,image_resolution,image_resolution],fill="white")
for point in startseeds:
    center_ellipse((point[0] + offset) * image_resolution / res, (point[1] + offset) * image_resolution / res,1,'red')
img.save("initial_seeds.png")

In [117]:
def replace_point(image):
    added = False
    while not added:
        rand_x = np.random.rand()
        rand_y = np.random.rand()
        img_x = rand_x * image_resolution
        img_y = rand_y * image_resolution
        pixel = np.max(image.getpixel((img_x,img_y)))

        rand_x = rand_x * res
        rand_y = rand_y * res
        
        if pixel < threshold:
            new_seed = np.array([rand_x,rand_y])
            added = True
            return new_seed
        else:
            continue

In [118]:
def Lloyd_Relax(seeds, res, image_path):

    # Creating voronoi diagram
    vor = Voronoi(seeds)

    #Dictionary for calculating centroids (regions corresp. to input points)
    region_dict = {}
    for point in vor.points:
        region_dict[str(point)] = []
        
    ##### CREATE KD TREE
    tree = KDTree(vor.points)

    ##### FINDING CLOSEST POINTS #####
    for x in range(res):
        for y in range(res):
            search = [x,y]
            closest = KD_closest_point(search,vor.points,tree) ##closest input point to each pixel
            region_dict[str(closest)].append([x,y])

    #### CALCULATING CENTROIDS #####
    centroids = []
    for point in vor.points:
        region = region_dict[str(point)]
        if len(region) > 0:
            # centroid = Weighted_Raster_Centroid(region,"weighted_test.png",res)
            centroid = Weighted_Raster_Centroid(region,image_path,res)
        else:
            centroid = point

        centroids.append(centroid)

    return centroids

Running the loop

In [119]:
for i in range(5):
    print(f"Iteration {i}")
    img = Image.new('RGB', (final_resolution, final_resolution))
    draw = ImageDraw.Draw(img)
    draw.rectangle([0,0,final_resolution,final_resolution],fill="white")
    for point in startseeds:
        center_ellipse((point[0] + offset) * final_resolution / res,(point[1] + offset) * final_resolution / res,1,'red')
    for point in seeds:
        center_ellipse((point[0] + offset) * final_resolution / res, (point[1] + offset) * final_resolution / res,2,'black')
    img.save(f"sequence/stipple_iteration_{i}.png")
    newseeds = Lloyd_Relax(seeds,res,image_path)
    seeds = newseeds


Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4


In [125]:
final_points = []
image = Image.open(image_path)
for point in seeds:
    val = image.getpixel((point[0],point[1]))
    print(val)
    # if val < 100:
    #     final_points.append(point)
# seeds = final_points

(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 255, 255)
(255, 25

Visualization

In [121]:

img = Image.new('RGB', (final_resolution, final_resolution))
draw = ImageDraw.Draw(img)
point_size = 4
draw.rectangle([0,0,final_resolution,final_resolution],fill="white")
# for point in startseeds:
#     center_ellipse((point[0] + offset) * final_resolution / res,(point[1] + offset) * final_resolution / res,3,'red')
for point in seeds:
    center_ellipse((point[0] + offset) * final_resolution / res, (point[1] + offset) * final_resolution / res,point_size,'black')
img.save("stipple_result_kd.png")

In [122]:
threshold = 255
image = Image.open(image_path)
draw = ImageDraw.Draw(image)
for x in range(image_resolution):
    for y in range(image_resolution):
        value = image.getpixel((x,y))
        if value < threshold:
            value = 0
        else:
            value = 255
        # print(value)
        center_rectangle(x,y,1,1,round(value))
image.save('thresh_test.png')

TypeError: '<' not supported between instances of 'tuple' and 'int'

In [ ]:
arr = [0, 1, 2, 3, 4]
print(arr)
arr[2] = 6
print(arr)

[0, 1, 2, 3, 4]
[0, 1, 6, 3, 4]
